In [1]:
import torch
from collections import OrderedDict
from codes.metrics import *
import torch.nn.functional as F

class DNN(torch.nn.Module):
    def __init__(self, layers, dropout=0.1):
        super(DNN, self).__init__()

        # parameters
        self.depth = len(layers) - 1

        # set up layer order dict
        self.activation = torch.nn.Tanh

        layer_list = list()
        for i in range(self.depth - 1):
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1],bias=True))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            layer_list.append(('dropout_%d' % i,torch.nn.Dropout(dropout)))

        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )

        # deploy layers
        self.layers = torch.nn.Sequential(OrderedDict(layer_list))

    def forward(self, x):
        out = self.layers(x)
        return out
    
def training(model,optimizer, max_epoch,B_0,B_1,bin_width,numBins,W,sigma,alpha,beta, print_epoch =50):
    for epoch in range(max_epoch+1):
        model.train()
        optimizer.zero_grad()
        
        h_pred = F.sigmoid(model(s))
        wsp = weighted_statistical_disparity_torch( h_pred.reshape(1,-1).t(), B_0, B_1, bin_width)

        loss = sigma*torch.sum((h_pred - s)**2) + alpha*wsp + beta*(torch.sum(torch.mm(W,h_pred.reshape(1,-1).t()))/W.shape[0])
        
        if epoch % print_epoch == 0 :
            result = evaluate(h_pred.detach().numpy(), y, s, prt, W_, numBins = numBins, bin_width = bin_width,print_flag=0)
            print("Epoch: %5d  || Loss: %.4f || prec: %.6f |  Rank corr: %.6f  | fp: %.6f | wsd: %.6f | wrd: %.6f )" % (epoch, loss.item(), result['precision'], result['corr'], result['fairperception'], result['wsd'] ,result['wrd']))
            
        loss.backward()
        optimizer.step()
        
    return h_pred.detach().numpy(), evaluate(h_pred.detach().numpy(), y, s, prt, W_,B_0=B_0.numpy(),B_1=B_1.numpy(), numBins = numBins, bin_width = bin_width,print_flag=0)

In [2]:
import sys
import numpy as np
import pandas as pd

from fairsearchcore.models import FairScoreDoc
import fairsearchcore as fsc
#from codes.alg import FPRank, FSPR_model, best_FPRank
from codes.metrics import evaluate
from codes.data_loader import load_data
import matplotlib.pyplot as plt

######################################################################
# Column names
######################################################################

data = 'credit_married_2'
prt_attr = 'Married'
class_attr = 'h_c'
score_attr = 's'

######################################################################
# Configuration parameters
######################################################################

numBins = 10
bin_width = 0.1
score_norm = 1

filename = 'data/' + data 
output_file = './results/' + data 

adj, data, W_, s, y, prt = load_data(filename, class_attr, score_attr, prt_attr, score_norm = score_norm)
B_0, B_1 = generate_bins_matrix(s,prt,numBins)
B_0 = torch.tensor(B_0).type(dtype = torch.float32)
B_1 = torch.tensor(B_1).type(dtype = torch.float32)
s = torch.tensor(s).type(dtype = torch.float32)
W = torch.tensor(W_).type(dtype = torch.float32) - torch.eye(adj.shape[0])

In [3]:
max_epoch = 1000
layers = [s.shape[0],256,256,256,s.shape[0]]
lr = 0.0001
weight_decay = 1e-5
alpha_lt = [1,2,3,4,5,10,20,30,40,50,100]
beta_lt = [1,2,3,4,5,10,20,30,40,50,100]
sigma=1
result_lt=[]
params = {'alpha': list(), 'beta': list()}
results = {'corr': list(), 'precision': list(), 'fairperception': list(), 'wsd': list(), 'wrd': list(), 'h': list()}

In [4]:
for i in range(len(beta_lt)):
    alpha = alpha_lt[i]
    for j in range(len(alpha_lt)):
        beta = beta_lt[j]
        params['alpha'].append(alpha)
        params['beta'].append(beta)
        print("Alpha: ", alpha, 'Beta: ', beta)
        model = DNN(layers, dropout=0.0)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        h, result = training(model,optimizer, max_epoch,B_0,B_1,bin_width,numBins,W,sigma,alpha,beta)
        print("Finished! Results: -->|| prec: %.6f |  Rank corr: %.6f  | fp: %.6f | wsd: %.6f | wrd: %.6f )" % (result['precision'], result['corr'], result['fairperception'], result['wsd'],result['wrd']))
        
        results['h'].append(h)
        results['corr'].append(result['corr'])
        results['precision'].append(result['precision'])
        results['fairperception'].append(result['fairperception'])
        results['wsd'].append(result['wsd'])
        results['wrd'].append(result['wrd'])
        
        result_lt.append(result)

Alpha:  1 Beta:  1
Epoch:     0  || Loss: 2756.1248 || prec: 0.779616 |  Rank corr: 0.002564  | fp: 0.499933 | wsd: 0.006329 | wrd: 0.101535 )
Epoch:    50  || Loss: 436.9651 || prec: 0.831610 |  Rank corr: 0.438228  | fp: 0.556833 | wsd: 0.025171 | wrd: 0.052880 )
Epoch:   100  || Loss: 24.3408 || prec: 0.866717 |  Rank corr: 0.924519  | fp: 0.556767 | wsd: 0.009238 | wrd: 0.019714 )
Epoch:   150  || Loss: 4.7124 || prec: 0.872329 |  Rank corr: 0.989775  | fp: 0.515267 | wsd: 0.007936 | wrd: 0.011961 )
Epoch:   200  || Loss: 1.5162 || prec: 0.873761 |  Rank corr: 0.998005  | fp: 0.494767 | wsd: 0.008012 | wrd: 0.010533 )
Epoch:   250  || Loss: 0.6113 || prec: 0.874341 |  Rank corr: 0.999477  | fp: 0.486333 | wsd: 0.007972 | wrd: 0.009938 )
Epoch:   300  || Loss: 0.2846 || prec: 0.874686 |  Rank corr: 0.999830  | fp: 0.481467 | wsd: 0.007988 | wrd: 0.009678 )
Epoch:   350  || Loss: 0.1479 || prec: 0.874880 |  Rank corr: 0.999936  | fp: 0.479467 | wsd: 0.008092 | wrd: 0.009442 )
Epoch: 

KeyboardInterrupt: 

In [ ]:
precision_lt =[]
fair_p =[]
wsd_lt = []
wrd_lt = []
for result in result_lt:
    precision_lt.append(result['precision'])
    fair_p.append(result['fairperception'])
    wsd_lt.append(result['wsd'])
    wrd_lt.append(result['wrd'])

In [ ]:
plt.plot(precision_lt[:11],wsd_lt[:11])
plt.show()

In [ ]:
plt.plot(precision_lt[:11],fair_p[:11])
plt.show()

In [ ]:
plt.plot(precision_lt[11:11+11],wsd_lt[11:11+11])
plt.show()

In [ ]:
from codes.alg import *
h,best_param,utils = best_FPRank(results, params, criterion = 'pmean', toRank=False)
result = evaluate(h, y, s, prt, W_,B_0=B_0.numpy(),B_1=B_1.numpy(), numBins = numBins, bin_width = bin_width,print_flag=0)
print("|| prec: %.6f |  Rank corr: %.6f  | fp: %.6f | wsd: %.6f | wrd: %.6f )" % (result['precision'], result['corr'], result['fairperception'], result['wsd'] ,result['wrd']))

In [ ]:
print(results)

In [ ]:
out_df = pd.DataFrame({'h': h, prt_attr : data[prt_attr]})
out_df.to_csv(output_file + '_DFR.csv', index = False)

In [ ]:
#best alpha 1 and beta 100